In [50]:
import pandas as pd
import statsmodels.stats.multitest as smm
import scipy.stats as sts
from statsmodels.sandbox.stats.multicomp import multipletests

from __future__ import division

In [2]:
data = pd.read_csv("gene_high_throughput_sequencing.csv")

Описание используемых данных

Данные для этой задачи взяты из исследования, проведенного в Stanford School of Medicine. В исследовании была предпринята попытка выявить набор генов, которые позволили бы более точно диагностировать возникновение рака груди на самых ранних стадиях.

В эксперименте принимали участие 24 человек, у которых не было рака груди (normal), 25 человек, у которых это заболевание было диагностировано на ранней стадии (early neoplasia), и 23 человека с сильно выраженными симптомами (cancer).

<img src="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/4X1xdyT5EeaOqxJVfwisfQ_68ac7a7a52dcac7f81505b3f20cd2896____________.png?expiry=1544486400000&hmac=HVGey8hIH0Df6ZNI9PZWsRJSAs1_WgFtY2oqpeVsPi8">

Ученые провели секвенирование биологического материала испытуемых, чтобы понять, какие из этих генов наиболее активны в клетках больных людей.

Секвенирование — это определение степени активности генов в анализируемом образце с помощью подсчёта количества соответствующей каждому гену РНК.

В данных для этого задания вы найдете именно эту количественную меру активности каждого из 15748 генов у каждого из 72 человек, принимавших участие в эксперименте.

Вам нужно будет определить те гены, активность которых у людей в разных стадиях заболевания отличается статистически значимо.

Кроме того, вам нужно будет оценить не только статистическую, но и практическую значимость этих результатов, которая часто используется в подобных исследованиях.

Диагноз человека содержится в столбце под названием "Diagnosis".

Практическая значимость изменения

Цель исследований — найти гены, средняя экспрессия которых отличается не только статистически значимо, но и достаточно сильно. В экспрессионных исследованиях для этого часто используется метрика, которая называется fold change (кратность изменения). Определяется она следующим образом:

F_c(C,T) = {T/C, T > C ; -C/T, T < C}

где C,T C, T C,T — средние значения экспрессии гена в control и treatment группах соответственно. По сути, fold change показывает, во сколько раз отличаются средние двух выборок.

In [3]:
data.Diagnosis.unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [26]:
normal = data[data.Diagnosis == "normal"].drop(["Patient_id", "Diagnosis"], axis=1)
early = data[data.Diagnosis == "early neoplasia"].drop(["Patient_id", "Diagnosis"], axis=1)
cancer = data[data.Diagnosis == "cancer"].drop(["Patient_id", "Diagnosis"], axis=1)

In [6]:
def student_test(data1, data2):
    students_mtest = list()
    genes = data1.columns.tolist()[2:]

    for gene in genes:
        students_mtest.append(sts.ttest_ind(data1[gene], data2[gene], equal_var=False)[1])
    
    # Gather only statisticaly significant diff genes and their pvalues.
    return [(gen,pval) for gen,pval in zip(genes, students_mtest)]

def write_answer(data, number):
    ans = "answer" + str(number) + ".txt"
    with open(ans, "w+") as file:
        file.write(str(data))
        

def fold_change(data1, data2, genes):
    data1 = data1[genes].mean()
    data2 = data2[genes].mean()

    pre_process = map(lambda t,c : t/c if t > c else -c/t, data1.values, data2.values)
    return [True if abs(i) > 1.5 else False for i in pre_process]

In [8]:
normal_early_test_genpval  = student_test(normal, early)
student_ne = [True if i[1] < 0.05 else False for i in normal_early_test_genpval]

write_answer(sum(student_ne), 1)

In [10]:
early_cancer_test_genpval = student_test(early, cancer)

student_ec = [True if i[1] < 0.05 else False for i in early_cancer_test_genpval]

print(sum(student_ec))
write_answer(sum(student_ec),2)

3490


In [47]:
normal_early_test_pval = [i[1] for i in normal_early_test_genpval]
early_cancer_test_pval= [i[1] for i in early_cancer_test_genpval]

In [12]:
reject_normal_early_holm = multipletests(normal_early_test_pval,alpha = 0.025, method = 'holm')[0]
reject_normal_early_holm_genes = [normal_early_test_genpval[i][0] for i in range(len(normal_early_test_pval)) if reject_normal_early_holm[i]]

reject_early_cancer_holm = multipletests(early_cancer_test_pval, alpha = 0.025, method = 'holm')[0]
reject_early_cancer_holm_genes = [early_cancer_test_genpval[i][0] for i in range(len(reject_early_cancer_holm)) if reject_early_cancer_holm[i]]

In [13]:
normal_early_holm = fold_change(normal, early, reject_normal_early_holm_genes)
early_cancer_holm = fold_change(early, cancer, reject_early_cancer_holm_genes)

write_answer(sum(normal_early_holm),3)
write_answer(sum(early_cancer_holm),4)

In [42]:
reject_normal_early_bh = multipletests(normal_early_test_pval, alpha = 0.025, method = 'fdr_bh')[0]
reject_normal_early_bh_genes = [normal_early_test_genpval[i][0] for i in range(len(normal_early_test_pval)) if reject_normal_early_bh[i]]

reject_early_cancer_bh = multipletests(early_cancer_test_pval, alpha = 0.025, method = 'fdr_bh')[0]
reject_early_cancer_bh_genes = [early_cancer_test_genpval[i][0] for i in range(len(reject_early_cancer_holm)) if reject_early_cancer_bh[i]]

In [45]:
normal_early_bh = fold_change(normal, early, reject_normal_early_bh_genes)
early_cancer_bh = fold_change(early, cancer, reject_early_cancer_bh_genes)

write_answer(sum(normal_early_bh),5)
write_answer(sum(early_cancer_bh),6)
